<a href="https://colab.research.google.com/github/VictoriaUsman/LLM-Engineering/blob/main/BeautifulSoup_%2B_OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [129]:
from openai import OpenAI
from google.colab import userdata
key = userdata.get('OPENAI_API_KEY')
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display




In [130]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

In [173]:
def Website(url):
  response = requests.get(url, headers=headers)
  soup = BeautifulSoup(response.content, 'html.parser')
  title = soup.title.string if soup.title else "No title found"
  for irrelevant in soup.body(["script", "style", "img", "input"]):
      irrelevant.decompose()
  text = soup.body.get_text(separator="\n", strip=True)
  return (title, text)


In [185]:
rappler = "https://www.rappler.com/"

In [186]:
title, text = Website(rappler)

In [187]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [189]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += text
    return user_prompt

In [ ]:
print(user_prompt_for(rappler))

In [119]:
def context_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [ ]:
context_for(rappler)

In [96]:
openai = OpenAI(api_key=key)

In [125]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = context_for(url)
    )
    return response.choices[0].message.content

In [192]:
summarize(rappler)

'# Summary of RAPPLER\n\nRAPPLER is a Philippine online news platform focusing on investigative journalism, civic engagement, and public interest stories. The site offers a range of content, including news updates, in-depth articles, and multimedia features covering local and international topics such as politics, business, technology, environment, and culture.\n\n## Latest News Highlights\n\n- **Typhoon Uwan Updates**: The typhoon has claimed at least 16 lives and left 2 missing in the Cordillera region, with ongoing concerns about secondary landslides in high-risk areas.\n  \n- **Super Typhoon Uwan and Typhoon Tino**: Live updates are being provided on both storms as they impact the Philippines, particularly with a focus on flooding and safety measures.\n\n- **Legislative Changes**: Senator Lacson has resumed his position as chair of the Senate blue ribbon committee.\n\n- **Heroic Act in Negros Occidental**: A man with special needs died while attempting to save others from floodwate

In [193]:

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [194]:
display_summary(rappler)

# Summary of RAPPLER

RAPPLER is a Philippine online news platform that emphasizes investigative journalism, civic engagement, and public interest. The website covers a broad range of topics including breaking news, in-depth analysis, and feature articles about local and world events. It hosts various segments such as **Business**, **Technology**, **Environment**, **Science**, and **Entertainment**, aiming to provide comprehensive coverage of occurrences both in the Philippines and globally.

## Latest News Highlights

- **Typhoon Uwan Impact**: At least 16 individuals are reported dead, and 2 are missing following Typhoon Uwan's intense winds and rain, raising concerns of potential landslides in the affected areas including Baguio City and Benguet.
  
- **Senate News**: Senator Lacson has resumed his role as the chair of the Senate Blue Ribbon Committee.

- **Fatal Flood Rescue**: A tragic incident in Negros Occidental reported a man with special needs who died while attempting to save others from floodwaters caused by Typhoon Tino.

- **COP30 Updates**: The UN Climate Change Conference (COP30) has commenced, focusing on climate finance and global cooperation for climate action amid the ongoing devastation in the Philippines from recent typhoons.

These updates reflect RAPPLER’s commitment to delivering timely and relevant news, particularly concerning natural disasters and civic matters, fostering informed engagement among its readership.